In [6]:
#rtWIZARD Protocol v1.0.1
#Will dispense sample liquid and running buffer into the same cassette well for 42 cassettes

#import opentrons.simulate if you want to simulate the protocol
#import opentrons.execute if you want to execute the protocol
import opentrons.simulate
protocol = opentrons.simulate.get_protocol_api('2.13')
protocol.home()

#assigns each OT2 slot a custom labware definition that corresponds to the cutout
#BEFORE EXECUTING: Run labware position check and copy/paste the Jupyter Notebook labware offsets as shown below:
'''
slot1 = protocol.load_labware("rtwizard_slots_147", 1)
slot1.set_offset(x=1.30, y=-3.50, z=-4.00)
'''
#paste offsets below:



slot1 = protocol.load_labware('rtwizard_slots_147', 1)
slot2 = protocol.load_labware('rtwizard_slots_258', 2)
slot3 = protocol.load_labware('rtwizard_slots_369', 3)
slot4 = protocol.load_labware('rtwizard_slots_147', 4)
slot5 = protocol.load_labware('rtwizard_slots_258', 5)
slot6 = protocol.load_labware('rtwizard_slots_369', 6)
slot7 = protocol.load_labware('rtwizard_slots_147', 7)
slot8 = protocol.load_labware('rtwizard_slots_258', 8)
slot9 = protocol.load_labware('rtwizard_slots_369', 9)

sample_plate = protocol.load_labware('rtwizard_48_wellplate_sarstedt_1100ul_tubes', 10)
tips = protocol.load_labware('opentrons_96_tiprack_20ul', 11)



#comment or uncomment depending on pipette setup
p300 = protocol.load_instrument('p300_single_gen2', 'left', tip_racks=[tips])
p20 = protocol.load_instrument('p20_single_gen2', 'right', tip_racks=[tips])

def main():

    #test-defined values
    #Adjust these values based on the lateral flow assay protocol
    
    #INGEZIM ASF CROM Ag
    num_wells = 1
    duo_config = False
    uses_buffer = True
    immediately_add_buffer = False
    leftmost_well_for_sample = True
    buffer_in_which_well = 1
    
    num_drops_sample = 1
    sample_vol = 20 #ul
    sample_vol_per_drop = 20

    num_drops_buffer = 3
    buffer_vol = 60
    buffer_vol_per_drop = 20


    #transfer sample liquid to cassettes
    transfer_samples(sample_vol)
    
    if(uses_buffer == True):
        #transfer X drops of running buffer to cassettes
        p20.pick_up_tip()
        for i in range(num_drops_buffer):
            transfer_buffer(buffer_vol_per_drop)
    
    
    #print console output
    for line in protocol.commands():
        print(line)

#fin
        
def transfer_samples(sample_vol):
    for i in range(5):

        #determining which slots to dispense on for this iteration of the loop
        if (i == 0 or i == 1):
            start_slot = slot1
            mid_slot = slot4
            fin_slot = slot7
        elif (i == 2):
            start_slot = slot2
            mid_slot = slot5
            fin_slot = slot8
        else:
            start_slot = slot3
            mid_slot = slot6
            fin_slot = slot9

        #loading_wells: list of wells being loaded with sample liquid
        if ((i % 2 == 0 and i != 4) or i == 3):
            loading_wells = ['C1', 'B1', 'A1']
            loading_wells2 = ['C1', 'B1']
        else:
            loading_wells = ['C2', 'B2', 'A2']
            loading_wells2 = ['C2', 'B2']
        
        #transfers
        row = sample_plate.rows()[i]
        p20.transfer(
            sample_vol, 
            row[:3], 
            [start_slot.wells_by_name()[well_name] for well_name in loading_wells],
            new_tip='always')
        p20.transfer(
            sample_vol, 
            row[3:6], 
            [mid_slot.wells_by_name()[well_name] for well_name in loading_wells],
            new_tip='always')
        p20.transfer(
            sample_vol, 
            row[6:8], 
            [fin_slot.wells_by_name()[well_name] for well_name in loading_wells2],
            new_tip='always')

    p20.transfer(
        sample_vol, 
        [sample_plate.wells_by_name()[well_name] for well_name in ['F1', 'F2']], 
        [slot7.wells_by_name()[well_name] for well_name in ['A1', 'A2']],
        new_tip='always')

def transfer_buffer(buffer_vol_per_drop):
    for i in range(5):
        
        #determining which slots to dispense on for this iteration of the loop
        if (i == 0 or i == 1):
            start_slot = slot1
            mid_slot = slot4
            fin_slot = slot7
        elif (i == 2):
            start_slot = slot2
            mid_slot = slot5
            fin_slot = slot8
        else:
            start_slot = slot3
            mid_slot = slot6
            fin_slot = slot9

        #loading_wells: list of wells being loaded with running buffer
        if ((i % 2 == 0 and i != 4) or i == 3):
            loading_wells = ['C1', 'B1', 'A1']
            loading_wells2 = ['C1', 'B1']
        else:
            loading_wells = ['C2', 'B2', 'A2']
            loading_wells2 = ['C2', 'B2']

        #listing wells F3-F8 which contain running buffer
        buffer_wells = ['F3', 'F4', 'F5', 'F6', 'F7', 'F8']
        
        #distributes
        #TODO: determine if disposal_volume should be increased for more consistent dispense volumes at the cost of time and buffer
        p20.distribute(
            buffer_vol_per_drop, 
            [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
            [start_slot.wells_by_name()[well_name] for well_name in loading_wells], 
            new_tip='never',
            disposal_volume=0)
        p20.distribute(
            buffer_vol_per_drop, 
            [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
            [mid_slot.wells_by_name()[well_name] for well_name in loading_wells],
            new_tip='never',
            disposal_volume=0)
        p20.distribute(
            buffer_vol_per_drop, 
            [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
            [fin_slot.wells_by_name()[well_name] for well_name in loading_wells2],
            new_tip='never',
            disposal_volume=0)

    p20.distribute(
        buffer_vol_per_drop, 
        [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
        [slot7.wells_by_name()[well_name] for well_name in ['A1', 'A2']],
        new_tip='never',
        disposal_volume=0)
    
if __name__ == "__main__":
    main()

C:\Users\bhinn\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\bhinn\.opentrons\deck_calibration.json not found. Loading defaults


Transferring 20.0 from A1 of rtWIZARD 48 Wellplate with Sarstedt 1100ul Tubes on 10 to C1 of rtWIZARD Slots 1, 4, 7 on 1
Picking up tip from A1 of Opentrons 96 Tip Rack 20 µL on 11
Aspirating 20.0 uL from A1 of rtWIZARD 48 Wellplate with Sarstedt 1100ul Tubes on 10 at 7.56 uL/sec
Dispensing 20.0 uL into C1 of rtWIZARD Slots 1, 4, 7 on 1 at 7.56 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from B1 of Opentrons 96 Tip Rack 20 µL on 11
Aspirating 20.0 uL from A2 of rtWIZARD 48 Wellplate with Sarstedt 1100ul Tubes on 10 at 7.56 uL/sec
Dispensing 20.0 uL into B1 of rtWIZARD Slots 1, 4, 7 on 1 at 7.56 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from C1 of Opentrons 96 Tip Rack 20 µL on 11
Aspirating 20.0 uL from A3 of rtWIZARD 48 Wellplate with Sarstedt 1100ul Tubes on 10 at 7.56 uL/sec
Dispensing 20.0 uL into A1 of rtWIZARD Slots 1, 4, 7 on 1 at 7.56 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 20.0 from A